In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score, f1_score, confusion_matrix
import json
import os
from metrics import *

In [2]:
MODEL_NAME = 'knn'

In [3]:
# Load features and labels
X = np.load('../../features.npy')
y = np.load('../../labels.npy')

X.shape, y.shape

((8724, 2048), (8724,))

In [4]:
# Count the number of samples in each class
print('Number of samples in each class:')
print(np.unique(y, return_counts=True))

# Define class names
class_names = ["Few", "Many", "None"]
print('Class names:')
print(class_names)

Number of samples in each class:
(array([0, 1, 2]), array([6232,  256, 2236]))
Class names:
['Few', 'Many', 'None']


In [5]:
# Divide the data into 2 classes only (Normal and Abnormal)
y_binary = np.zeros(y.shape)
y_binary[y != 2] = 0 # Few and Many
y_binary[y == 2] = 1 # None

# Check the number of samples in each class
print('Number of samples in each class:')
print(np.unique(y_binary, return_counts=True))

# Define the new class names
binary_class_names = ["Abormal", "Normal"]
print('Binary class names:')
print(binary_class_names)

Number of samples in each class:
(array([0., 1.]), array([6488, 2236]))
Binary class names:
['Abormal', 'Normal']


In [6]:
# Define the parameter grid for KNN
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'p': [1, 2],  # Only relevant for Minkowski metric
    'leaf_size': [10, 20, 30, 40, 50],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

custom_score = custom_score = make_scorer(custom_scorer, greater_is_better=True, 
                           response_method='predict_proba'
                           )

In [9]:
# Initialize the KNN classifier
knn = KNeighborsClassifier()

# Define the cross-validation strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=knn, 
    param_distributions=param_grid,
    n_iter=100, 
    cv=cv, 
    verbose=2, 
    random_state=42, 
    n_jobs=-1,
    scoring=custom_score
)

In [8]:
random_search.fit(X, y_binary)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END algorithm=brute, leaf_size=50, metric=euclidean, n_neighbors=5, p=2, weights=distance; total time=   2.1s
[CV] END algorithm=brute, leaf_size=50, metric=euclidean, n_neighbors=5, p=2, weights=distance; total time=   2.1s
[CV] END algorithm=brute, leaf_size=50, metric=euclidean, n_neighbors=5, p=2, weights=distance; total time=   2.2s
[CV] END algorithm=brute, leaf_size=50, metric=euclidean, n_neighbors=5, p=2, weights=distance; total time=   2.3s
[CV] END algorithm=brute, leaf_size=50, metric=euclidean, n_neighbors=5, p=2, weights=distance; total time=   2.3s
[CV] END algorithm=ball_tree, leaf_size=10, metric=minkowski, n_neighbors=5, p=2, weights=uniform; total time=  17.4s
[CV] END algorithm=ball_tree, leaf_size=10, metric=minkowski, n_neighbors=5, p=2, weights=uniform; total time=  17.5s
[CV] END algorithm=ball_tree, leaf_size=10, metric=minkowski, n_neighbors=5, p=2, weights=uniform; total time=  17.5s
[CV] END

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=KNeighborsClassifier(), n_iter=100, n_jobs=-1,
                   param_distributions={'algorithm': ['auto', 'ball_tree',
                                                      'kd_tree', 'brute'],
                                        'leaf_size': [10, 20, 30, 40, 50],
                                        'metric': ['euclidean', 'manhattan',
                                                   'minkowski'],
                                        'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
                                        'p': [1, 2],
                                        'weights': ['uniform', 'distance']},
                   random_state=42,
                   scoring=make_scorer(custom_scorer, response_method='predict_proba'),
                   verbose=2)

In [9]:
# Save best parameters
with open('dd_normal_vs_abnormal_best_params.json', 'w') as f:
    json.dump(random_search.best_params_, f, indent=4)

In [7]:
# # Get the best parameters
# best_params = random_search.best_params_
# print("Best parameters:", best_params)

# Load the best parameters from a json file
with open('dd_normal_vs_abnormal_best_params.json', 'r') as f:
    best_params = json.load(f)

# Now, let's perform a final 10-fold cross-validation using the best model
best_rf_normal_abnormal = KNeighborsClassifier(**best_params)

In [11]:
fold_results = []

for fold, (train_index, val_index) in enumerate(cv.split(X, y_binary), 1):
    print(f"Fold {fold}")
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y_binary[train_index], y_binary[val_index]
    
    best_rf_normal_abnormal.fit(X_train, y_train)
    y_pred = best_rf_normal_abnormal.predict(X_val)
    y_prob = best_rf_normal_abnormal.predict_proba(X_val)
    
    # Calculate metrics
    accuracy, class_metrics, auc, f1, cm, avg_sensitivity, avg_specificity = calculate_metrics(y_val, y_pred, y_prob, num_classes=2)

    # Log metrics for this fold
    metrics = {
        'fold': fold,
        'val_accuracy': accuracy,
        'val_auc': auc,
        'val_f1': f1,
        'avg_sensitivity': avg_sensitivity,
        'avg_specificity': avg_specificity,
        **{f'class_{binary_class_names[i]}_sensitivity': metrics["sensitivity"] for i, metrics in enumerate(class_metrics)},
        **{f'class_{binary_class_names[i]}_specificity': metrics["specificity"] for i, metrics in enumerate(class_metrics)},
        **{f'class_{binary_class_names[i]}_f1': 2 * metrics["sensitivity"] * metrics["specificity"] / (metrics["sensitivity"] + metrics["specificity"]) for i, metrics in enumerate(class_metrics)},
    }
    
    custom_log(metrics, model_name=f'dd_{MODEL_NAME}_nor_abn', log_dir='logs_dd/nor_vs_abn/')
    print("Metrics for this fold:")
    print(metrics)

    # Plot confusion matrix for this fold
    plot_confusion_matrix(cm, class_names=binary_class_names, epoch_num=0, model_name=f'dd_{MODEL_NAME}_nor_abn', fold_num=fold, save_dir='figures_dd/nor_vs_abn/')
    
    fold_results.append(metrics)

# Calculate and print average results across all folds
avg_results = {key: np.mean([fold[key] for fold in fold_results if key in fold]) 
               for key in fold_results[0].keys() if key != 'fold'}

print("Average results across all folds:")
for key, value in avg_results.items(): 
    print(f"{key}: {value}")

# Log average results
custom_log(avg_results, model_name=f'dd_{MODEL_NAME}_average_nor_abn', log_dir='logs_dd/nor_vs_abn/')

Fold 1
Metrics for this fold:
{'fold': 1, 'val_accuracy': 0.974785100286533, 'val_auc': 0.9871959269638715, 'val_f1': 0.9496567505720824, 'avg_sensitivity': 0.95958332040689, 'avg_specificity': 0.95958332040689, 'class_Abormal_sensitivity': 0.9907550077041603, 'class_Normal_sensitivity': 0.9284116331096197, 'class_Abormal_specificity': 0.9284116331096197, 'class_Normal_specificity': 0.9907550077041603, 'class_Abormal_f1': 0.958570720387387, 'class_Normal_f1': 0.958570720387387}
Fold 2
Metrics for this fold:
{'fold': 2, 'val_accuracy': 0.9736389684813753, 'val_auc': 0.9897545009875802, 'val_f1': 0.9483146067415731, 'avg_sensitivity': 0.9639464259245853, 'avg_specificity': 0.9639464259245853, 'class_Abormal_sensitivity': 0.9838212634822804, 'class_Normal_sensitivity': 0.9440715883668904, 'class_Abormal_specificity': 0.9440715883668904, 'class_Normal_specificity': 0.9838212634822804, 'class_Abormal_f1': 0.9635366426033123, 'class_Normal_f1': 0.9635366426033123}
Fold 3
Metrics for this fol

In [12]:
# Predict the class Abnormal for each sample using the best model from the first stage
y_pred = best_rf_normal_abnormal.predict(X)
abnormal_indices = np.where(y_pred == 0)[0]
print(f"Number of predicted abnormal samples: {len(abnormal_indices)}")

Number of predicted abnormal samples: 6495


In [13]:
# Filter the dataset to include only the predicted Abnormal sample
X_pred_abnormal = X[abnormal_indices]
y_pred_abnormal = y[abnormal_indices]

# Check the number of samples in each class
print('Number of samples in each class:')
len(y_pred_abnormal), len(X_pred_abnormal)

Number of samples in each class:


(6495, 6495)

In [14]:
print(np.unique(y_pred_abnormal, return_counts=True))
normal_in_abnormal = y_pred_abnormal == 2  # 2 is the label for Normal
len(y_pred_abnormal[normal_in_abnormal])

(array([0, 1, 2]), array([6219,  254,   22]))


22

In [15]:
X_abnormal = X_pred_abnormal[~normal_in_abnormal]
y_abnormal = y_pred_abnormal[~normal_in_abnormal]

# Check the number of samples in each class
print('Number of samples in each class:')
np.unique(y_abnormal, return_counts=True)

Number of samples in each class:


(array([0, 1]), array([6219,  254]))

In [16]:
# Initialize RandomizedSearchCV
random_search_abn = RandomizedSearchCV(
    estimator=knn, 
    param_distributions=param_grid,
    n_iter=100, 
    cv=cv, 
    verbose=2, 
    random_state=42, 
    n_jobs=-1,
    scoring=custom_score
)

In [22]:
random_search_abn.fit(X_abnormal, y_abnormal)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END algorithm=brute, leaf_size=50, metric=euclidean, n_neighbors=5, p=2, weights=distance; total time=   1.3s
[CV] END algorithm=brute, leaf_size=50, metric=euclidean, n_neighbors=5, p=2, weights=distance; total time=   1.3s
[CV] END algorithm=brute, leaf_size=50, metric=euclidean, n_neighbors=5, p=2, weights=distance; total time=   1.4s
[CV] END algorithm=brute, leaf_size=50, metric=euclidean, n_neighbors=5, p=2, weights=distance; total time=   1.4s
[CV] END algorithm=brute, leaf_size=50, metric=euclidean, n_neighbors=5, p=2, weights=distance; total time=   1.4s
[CV] END algorithm=ball_tree, leaf_size=10, metric=minkowski, n_neighbors=5, p=2, weights=uniform; total time=  10.4s
[CV] END algorithm=ball_tree, leaf_size=10, metric=minkowski, n_neighbors=5, p=2, weights=uniform; total time=  10.5s
[CV] END algorithm=ball_tree, leaf_size=10, metric=minkowski, n_neighbors=5, p=2, weights=uniform; total time=  10.7s
[CV] END

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=KNeighborsClassifier(), n_iter=100, n_jobs=-1,
                   param_distributions={'algorithm': ['auto', 'ball_tree',
                                                      'kd_tree', 'brute'],
                                        'leaf_size': [10, 20, 30, 40, 50],
                                        'metric': ['euclidean', 'manhattan',
                                                   'minkowski'],
                                        'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
                                        'p': [1, 2],
                                        'weights': ['uniform', 'distance']},
                   random_state=42,
                   scoring=make_scorer(custom_scorer, response_method='predict_proba'),
                   verbose=2)

In [23]:
# Save best parameters few vs many
with open('dd_few_vs_many_best_params.json', 'w') as f:
    json.dump(random_search_abn.best_params_, f, indent=4)

In [17]:
# Load the best parameters from a json file
with open('dd_few_vs_many_best_params.json', 'r') as f:
    best_params_abn = json.load(f)

best_rf_few_many = KNeighborsClassifier(**best_params_abn)

abn_class_names = ["Few", "Many"]

In [18]:
fold_results = []

for fold, (train_index, val_index) in enumerate(cv.split(X_abnormal, y_abnormal), 1):
    print(f"Fold {fold}")
    X_train, X_val = X_abnormal[train_index], X_abnormal[val_index]
    y_train, y_val = y_abnormal[train_index], y_abnormal[val_index]
    
    best_rf_few_many.fit(X_train, y_train)
    y_pred = best_rf_few_many.predict(X_val)
    y_prob = best_rf_few_many.predict_proba(X_val)
    
    # Calculate metrics
    accuracy, class_metrics, auc, f1, cm, avg_sensitivity, avg_specificity = calculate_metrics(y_val, y_pred, y_prob, num_classes=2)

    # Log metrics for this fold
    metrics = {
        'fold': fold,
        'val_accuracy': accuracy,
        'val_auc': auc,
        'val_f1': f1,
        'avg_sensitivity': avg_sensitivity,
        'avg_specificity': avg_specificity,
        **{f'class_{abn_class_names[i]}_sensitivity': metrics["sensitivity"] for i, metrics in enumerate(class_metrics)},
        **{f'class_{abn_class_names[i]}_specificity': metrics["specificity"] for i, metrics in enumerate(class_metrics)},
        **{f'class_{abn_class_names[i]}_f1': 2 * metrics["sensitivity"] * metrics["specificity"] / (metrics["sensitivity"] + metrics["specificity"]) for i, metrics in enumerate(class_metrics)},
    }
    
    custom_log(metrics, model_name=f'dd_{MODEL_NAME}_few_many', log_dir='logs_dd/few_vs_many/')
    print("Metrics for this fold:")
    print(metrics)

    # Plot confusion matrix for this fold
    plot_confusion_matrix(cm, class_names=abn_class_names, epoch_num=0, model_name=f'dd_{MODEL_NAME}_few_many', fold_num=fold, save_dir='figures_dd/few_vs_many/')
    
    fold_results.append(metrics)

# Calculate and print average results across all folds
avg_results = {key: np.mean([fold[key] for fold in fold_results if key in fold]) 
               for key in fold_results[0].keys() if key != 'fold'}

print("Average results across all folds:")
for key, value in avg_results.items(): 
    print(f"{key}: {value}")

# Log average results
custom_log(avg_results, model_name=f'dd_{MODEL_NAME}_few_many_average', log_dir='logs_dd/few_vs_many/')

Fold 1
Metrics for this fold:
{'fold': 1, 'val_accuracy': 0.9969111969111969, 'val_auc': 0.9800059895340772, 'val_f1': 0.9591836734693877, 'avg_sensitivity': 0.9607843137254901, 'avg_specificity': 0.9607843137254901, 'class_Few_sensitivity': 1.0, 'class_Many_sensitivity': 0.9215686274509803, 'class_Few_specificity': 0.9215686274509803, 'class_Many_specificity': 1.0, 'class_Few_f1': 0.9591836734693878, 'class_Many_f1': 0.9591836734693878}
Fold 2
Metrics for this fold:
{'fold': 2, 'val_accuracy': 0.9992277992277993, 'val_auc': 0.9900542210453314, 'val_f1': 0.9900990099009901, 'avg_sensitivity': 0.9901960784313726, 'avg_specificity': 0.9901960784313726, 'class_Few_sensitivity': 1.0, 'class_Many_sensitivity': 0.9803921568627451, 'class_Few_specificity': 0.9803921568627451, 'class_Many_specificity': 1.0, 'class_Few_f1': 0.99009900990099, 'class_Many_f1': 0.99009900990099}
Fold 3
Metrics for this fold:
{'fold': 3, 'val_accuracy': 0.9992277992277993, 'val_auc': 0.9998502616480677, 'val_f1': 0

In [19]:
def combine_predictions(normal_abnormal_pred, few_many_pred):
    # Initialize final_pred with the same shape as normal_abnormal_pred, filled with 2 (Normal)
    final_pred = np.full(normal_abnormal_pred.shape, 2)
    
    # Create a mask where normal_abnormal_pred is 0 (Abnormal)
    abnormal_mask = normal_abnormal_pred == 0
    
    # Update final_pred where the mask is True with values from few_many_pred
    final_pred[abnormal_mask] = few_many_pred
    
    # Return the final combined predictions
    return final_pred

def combine_probabilities(normal_abnormal_prob, few_many_prob, abnormal_mask):
    # Initialize with probabilities for Normal class
    combined_prob = np.column_stack((np.zeros_like(normal_abnormal_prob[:, 0]), 
                                     np.zeros_like(normal_abnormal_prob[:, 0]), 
                                     normal_abnormal_prob[:, 1]))
    
    # Update probabilities for Abnormal (Few and Many) classes
    if few_many_prob.size > 0:
        combined_prob[abnormal_mask, 0] = few_many_prob[:, 0] * normal_abnormal_prob[abnormal_mask, 0]
        combined_prob[abnormal_mask, 1] = few_many_prob[:, 1] * normal_abnormal_prob[abnormal_mask, 0]
        combined_prob[abnormal_mask, 2] = 0  # Probability of being Normal for predicted Abnormal samples
    
    # Normalize probabilities
    row_sums = combined_prob.sum(axis=1)
    combined_prob /= row_sums[:, np.newaxis]
    
    return combined_prob

In [20]:
overall_true = []
overall_pred = []
overall_prob = []

for fold, (train_index, val_index) in enumerate(cv.split(X, y), 1):
    print(f"Fold {fold}")
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    # Stage 1: Normal vs Abnormal
    y_train_binary = (y_train == 2).astype(int)
    best_rf_normal_abnormal.fit(X_train, y_train_binary)
    y_pred_binary = best_rf_normal_abnormal.predict(X_val)
    y_prob_binary = best_rf_normal_abnormal.predict_proba(X_val)
    
    # Stage 2: Few vs Many (only for predicted Abnormal samples)
    abnormal_mask_train = y_train != 2
    X_train_abnormal = X_train[abnormal_mask_train]
    y_train_abnormal = y_train[abnormal_mask_train]
    
    best_rf_few_many.fit(X_train_abnormal, y_train_abnormal)
    
    abnormal_mask_val = y_pred_binary == 0
    X_val_abnormal = X_val[abnormal_mask_val]
    
    if len(X_val_abnormal) > 0:
        y_pred_few_many = best_rf_few_many.predict(X_val_abnormal)
        y_prob_few_many = best_rf_few_many.predict_proba(X_val_abnormal)
    else:
        y_pred_few_many = np.array([])
        y_prob_few_many = np.array([])
    
    # Combine predictions and probabilities
    y_pred_combined = combine_predictions(y_pred_binary, y_pred_few_many)
    y_prob_combined = combine_probabilities(y_prob_binary, y_prob_few_many, abnormal_mask_val)
    
    overall_true.extend(y_val)
    overall_pred.extend(y_pred_combined)
    overall_prob.extend(y_prob_combined)
    
    # Calculate and log metrics for this fold
    accuracy, class_metrics, auc, f1, cm, avg_sensitivity, avg_specificity = calculate_metrics(y_val, y_pred_combined, y_prob_combined, num_classes=3)
     
    metrics = {
        'fold': fold,
        'val_accuracy': accuracy,
        'val_auc': auc,
        'val_f1': f1,
        'avg_sensitivity': avg_sensitivity,
        'avg_specificity': avg_specificity,
        **{f'class_{class_names[i]}_sensitivity': metrics["sensitivity"] for i, metrics in enumerate(class_metrics)},
        **{f'class_{class_names[i]}_specificity': metrics["specificity"] for i, metrics in enumerate(class_metrics)},
        **{f'class_{class_names[i]}_f1': 2 * metrics["sensitivity"] * metrics["specificity"] / (metrics["sensitivity"] + metrics["specificity"]) for i, metrics in enumerate(class_metrics)},
    }
    
    custom_log(metrics, model_name=f'dd_{MODEL_NAME}_combined', log_dir='logs_dd/combined/')
    print("Metrics for this fold:")
    print(metrics)
    
    # Plot confusion matrix for this fold
    plot_confusion_matrix(cm, class_names=class_names, epoch_num=0, model_name=f'dd_{MODEL_NAME}_combined', fold_num=fold, save_dir='figures_dd/combined/')

# Calculate overall metrics
overall_true = np.array(overall_true)
overall_pred = np.array(overall_pred)
overall_prob = np.array(overall_prob)

accuracy, class_metrics, auc, f1, cm, avg_sensitivity, avg_specificity = calculate_metrics(overall_true, overall_pred, overall_prob, num_classes=3)

overall_metrics = {
    'overall_accuracy': accuracy,
    'overall_auc': auc,
    'overall_f1': f1,
    'overall_avg_sensitivity': avg_sensitivity,
    'overall_avg_specificity': avg_specificity,
    **{f'overall_class_{class_names[i]}_sensitivity': metrics["sensitivity"] for i, metrics in enumerate(class_metrics)},
    **{f'overall_class_{class_names[i]}_specificity': metrics["specificity"] for i, metrics in enumerate(class_metrics)},
    **{f'overall_class_{class_names[i]}_f1': 2 * metrics["sensitivity"] * metrics["specificity"] / (metrics["sensitivity"] + metrics["specificity"]) for i, metrics in enumerate(class_metrics)},
}

print("Overall metrics:")
for key, value in overall_metrics.items():
    print(f"{key}: {value}")

custom_log(overall_metrics, model_name=f'dd_{MODEL_NAME}_combined_overall', log_dir='logs_dd/combined/')

# Plot overall confusion matrix
plot_confusion_matrix(cm, class_names=class_names, epoch_num=0, model_name=f'dd_{MODEL_NAME}_combined_overall', save_dir='figures_dd/combined/') 

Fold 1
Metrics for this fold:
{'fold': 1, 'val_accuracy': 0.9879656160458453, 'val_auc': 0.9853301497875423, 'val_f1': 0.9879579606080813, 'avg_sensitivity': 0.9891240842301913, 'avg_specificity': 0.9901291945307761, 'class_Few_sensitivity': 0.991980753809142, 'class_Many_sensitivity': 1.0, 'class_None_sensitivity': 0.9753914988814317, 'class_Few_specificity': 0.9779116465863453, 'class_Many_specificity': 0.999409681227863, 'class_None_specificity': 0.9930662557781201, 'class_Few_f1': 0.9848959589312646, 'class_Many_f1': 0.9997047534691467, 'class_None_f1': 0.9841495266221935}
Fold 2
Metrics for this fold:
{'fold': 2, 'val_accuracy': 0.9810888252148997, 'val_auc': 0.9785507226456343, 'val_f1': 0.9811134544498447, 'avg_sensitivity': 0.9844811739076095, 'avg_specificity': 0.9858099143283022, 'class_Few_sensitivity': 0.9847634322373697, 'class_Many_sensitivity': 1.0, 'class_None_sensitivity': 0.9686800894854586, 'class_Few_specificity': 0.9718875502008032, 'class_Many_specificity': 0.9994